In [1]:
from openai import OpenAI
#temp key:
openai_api_key = "sk-proj-"
client = OpenAI(api_key=openai_api_key)

In [27]:
import json
from datetime import datetime


def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }


developer_prompt = f"""
You're an asistant of a courier in a food delivery company. Today is ${str(datetime.now())}.
You're given a question from a courier and your task is to answer it. 

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]


known_courier_data = {
    'first_name': "John",
    'last_name': "Doe",
    'date_of_birth': "05.11.1995",
    'vehicle_type': "Car",
    'work_shifts': {
        'monday': {'from_hour': 8, 'to_hour': 12},
        'wednesday': {'from_hour': 12, 'to_hour': 18},
        'friday': {'from_hour': 10, 'to_hour': 15}
    },
    'notes': [
    ]
}


def get_courier_profile() -> dict:
    return known_courier_data

def update_courier_profile(first_name:str = None, last_name:str = None, date_of_birth:str = None, vehicle_type:str = None) -> None:
    if (first_name != None): known_courier_data['first_name'] = first_name.strip()
    if (last_name != None): known_courier_data['last_name'] = last_name.strip()
    if (date_of_birth != None): known_courier_data['date_of_birth'] = date_of_birth.strip().lower()
    if (vehicle_type != None): known_courier_data['vehicle_type'] = vehicle_type.strip().lower()    
    return 'OK'

def update_courier_work_shifts(day_of_week:str = None, from_hour:int = None, to_hour:int = None) -> None:
    known_courier_data['work_shifts'][day_of_week.strip().lower()] = {'from_hour': from_hour, 'to_hour': to_hour}
    return 'OK'

def add_notes_about_courier(note:str) -> None:
    known_courier_data['notes'].append(note.strip())
    return 'OK'

# def get_time_and_date() -> str:
#     return str(datetime.now())

set_courier_first_name_tool = {
    "type": "function",
    "name": "update_courier_profile",
    "description": "Update courier profile.",
    "parameters": {
        "type": "object",
        "properties": {
            "first_name": {
                "type": "string",
                "description": "The first name of the courier."
            },
            "last_name": {
                "type": "string",
                "description": "The last name of the courier."
            },
            "date_of_birth": {
                "type": "string",
                "description": "The date of birth of the courier in following format YYYY-MM-DD."
            },
            "vehicle_type": {
                "type": "string",
                "description": "The prefered vehicle of the courier for deliveries. Can be only car or bike."
            }
        },
        "required": [],
        "additionalProperties": False
    }
}

update_courier_work_shifts_tool = {
    "type": "function",
    "name": "update_courier_work_shifts",
    "description": "Update courier shifts.",
    "parameters": {
        "type": "object",
        "properties": {
            "day_of_week": {
                "type": "string",
                "description": "The day of the week of the courier's shift."
            },
            "from_hour": {
                "type": "number",
                "description": "The hour when the courier starts."
            },
            "to_hour": {
                "type": "number",
                "description": "The hour when the courier ends."
            },
        },
        "required": ['day_of_week', 'from_hour', 'to_hour'],
        "additionalProperties": False
    }
}

add_courier_note_tool = {
    "type": "function",
    "name": "add_notes_about_courier",
    "description": "Adds additional notes to the profile of the courier.",
    "parameters": {
        "type": "object",
        "properties": {
            "note": {
                "type": "string",
                "description": "The extra information about the courier that is not yet part of the profile."
            }
        },
        "required": ["note"],
        "additionalProperties": False
    }
}

get_courier_profile_tool = {
    "type": "function",
    "name": "get_courier_profile",
    "description": "Get courier profile.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    }
}

# get_time_and_date_tool = {
#     "type": "function",
#     "name": "get_time_and_date",
#     "description": "Get current time and date to have a time referance.",
#     "parameters": {
#         "type": "object",
#         "properties": {},
#         "required": [],
#         "additionalProperties": False
#     }
# }

tools = [get_courier_profile_tool, set_courier_first_name_tool, update_courier_work_shifts_tool, add_courier_note_tool]

while True:
    question = input() 
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: 
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break


 who am i


function_call: ResponseFunctionToolCall(arguments='{}', call_id='call_Zazzxq4GJtrjNkAMWOprQVU4', name='get_courier_profile', type='function_call', id='fc_687992981f64819c90d3a95d2b6570a302797d7ad391bc94', status='completed')

You are John Doe, born on November 5, 1995. You primarily use a car for your deliveries. Your work shifts are as follows:

- **Monday**: 8 AM to 12 PM
- **Wednesday**: 12 PM to 6 PM
- **Friday**: 10 AM to 3 PM

Would you like to update any of your profile details or work shifts?



 my first name is Tom 


function_call: ResponseFunctionToolCall(arguments='{"first_name":"Tom"}', call_id='call_D93Cb3xkGBhcmoURIavMmPck', name='update_courier_profile', type='function_call', id='fc_687992bf4bd0819ca2c77fcd6e96c64802797d7ad391bc94', status='completed')

Your first name has been updated to Tom. 

Is there anything else you would like to change or any other assistance you need?



 I don't have a driving licence


...

 who am I


function_call: ResponseFunctionToolCall(arguments='{}', call_id='call_6RHO8SLeNbM1GTynfz9nenLd', name='get_courier_profile', type='function_call', id='fc_6879933c8ab4819cb1a84bd3cc49f22f02797d7ad391bc94', status='completed')

You are Tom Doe, born on November 5, 1995. You use a bike for your deliveries. Your current work shifts are:

- **Monday**: 8 AM to 12 PM
- **Wednesday**: 12 PM to 6 PM
- **Friday**: 10 AM to 3 PM
- **Sunday**: 8 AM to 6 PM

Additionally, you do not have a driving license and are allergic to milk.

Is there anything else you would like to discuss or change?



 I was born exactly 20 years ago


function_call: ResponseFunctionToolCall(arguments='{"date_of_birth":"2005-07-18"}', call_id='call_LiupppEdQaY8Hn78tvAu0FGE', name='update_courier_profile', type='function_call', id='fc_6879938066f8819cb160c24f6c7049a602797d7ad391bc94', status='completed')

Your date of birth has been updated to July 18, 2005. 

Is there anything else you would like to update or any other questions you have?



 stop


In [28]:
known_courier_data

{'first_name': 'Tom',
 'last_name': 'Doe',
 'date_of_birth': '2005-07-18',
 'vehicle_type': 'bike',
 'work_shifts': {'monday': {'from_hour': 8, 'to_hour': 12},
  'wednesday': {'from_hour': 12, 'to_hour': 18},
  'friday': {'from_hour': 10, 'to_hour': 15},
  'sunday': {'from_hour': 8, 'to_hour': 18}},
 'notes': ['Tom does not have a driving license.', 'Tom is allergic to milk.']}